In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from spotipy.oauth2 import SpotifyOAuth
import requests
import pandas as pd
import time as tm
import os
# 🔹 Configurar credenciales de Spotify
client_id = "cb49cc625e6345c1a4414a0067fe80e2"
client_secret = "b7ab4bce9bcb481cb5e5b7cf7af58792"
lastfm_api_key = "20737815e90d55fca18923d9df9874c8"
lastfm_secret = "dc68775665a6481a3c42d0770ca4de66"
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Configurar API Key de Last.fm
LASTFM_API_KEY = lastfm_api_key

# Función para obtener el género de una canción
def get_genre_from_lastfm(song_name, artist_name):
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "track.gettoptags",
        "api_key": LASTFM_API_KEY,
        "artist": artist_name,
        "track": song_name,
        "format": "json"
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    # Extraer géneros si existen
    time.sleep(0.045)
    if response.status_code == 200:
        try:
            response = requests.get(url, params=params, timeout=5)  # Establece un tiempo de espera de 5 segundos
            response.raise_for_status()  # Lanza un error si el código de estado no es 200
            data = response.json()

            # Espera breve para evitar límites de tasa de la API
            time.sleep(0.125)

            if "toptags" in data and "tag" in data["toptags"]:
                genres = [tag["name"] for tag in data["toptags"]["tag"][:5]]  # Tomar los 5 principales
                return ", ".join(genres) if genres else "Unknown"
            return "Unknown"

        except requests.exceptions.RequestException as e:
            print(f"Error en la solicitud para {song_name} - {artist_name}: {e}")
            return "Error"

        except ValueError:
            print(f"Error al analizar JSON para {song_name} - {artist_name}")
            return "Error"

        except Exception as e:
                print(f"Error inesperado para {song_name} - {artist_name}: {e}")
                return "Error"

    else:
        return "NoConsultado"

# Función para obtener géneros de una lista de canciones



def load_pop_datasets(folder_path):
    dfs = []
    filenames = []
    for filename in os.listdir(folder_path):
        if "pop" in filename and filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dfs.append(df)
            filenames.append(filename)
    return dfs, filenames

# Load datasets from the specified folder
folder_path = 'datasets_ignore/'
dfs, names = load_pop_datasets(folder_path)



In [9]:
for df in dfs:
    df['genres'] = ""

dfs[0]

,artista,nombre_cancion,genres
0,daddy yankee,con calma,
1,post malone,wow.,
2,niykee heaton,bad guy,
3,post malone,sunflower - spider-man: into the spider-verse,
4,jonas brothers,sucker,
...,...,...,...
195,dean lewis,7 minutes,
196,schoolboy q,numb numb juice,
197,summer walker,girls need love (with drake) - remix,
198,brytiago,controla,


In [11]:
import time as tm

def update_genres_in_df(input_df, name, tries=0, n=10000, time_limit=3600*2):
    
    c = 0
    total_time = 0
    test_df = input_df.iloc[tries*n:(tries+1)*n].copy()

    for index, row in test_df.iterrows():
        try:
            c += 1
            start_time = tm.time()  # Tiempo de inicio de la iteración
            
            artist_name = str(row['artista']).split(",")[0].replace("[", "").replace("]", "").replace("'", "")
            genres = get_genre_from_lastfm(row['nombre_cancion'], artist_name)
            test_df.at[index, 'genres'] = genres
            print(c)
            
            end_time = tm.time()  # Tiempo de finalización de la iteración
            iteration_time = end_time - start_time
            total_time += iteration_time
            
            if total_time > time_limit:  # Si se superan los 3600 segundos (1 hora)
                print("Tiempo límite excedido. Guardando el DataFrame parcial.")
                break
        except Exception as e:
            print(f"Error occurred: {e}. Guardando el DataFrame parcial.")
            print("genres: ", genres)
            test_df.to_csv(f"{name}_{tries}.csv", index=False)
            break

    test_df.to_csv(f"{name}_genres_{tries}.csv", index=False)
    return test_df

# Uso de la función

for indice, df in enumerate(dfs):
    updated_df = update_genres_in_df(df, names[indice])
    print(updated_df)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
            artista                                 nombre_cancion  \
0      daddy yankee                                      con calma   
1       post malone                                           wow.   
2     niykee heaton                                        bad guy   
3       post malone  sunflow

In [ ]:
save_df = test_df[test_df['genres'].apply(lambda x: x != [])]
save_df.to_csv(f"lastfm_genres_{tries}.csv", index=False)


In [6]:
save_df1 = pd.read_csv(f"lastfm_genres_2024_0.csv")
save_df2 = pd.read_csv(f"lastfm_genres_2024_1.csv")
save_df3 = pd.read_csv(f"lastfm_genres_2024_2.csv")
save_df4 = pd.read_csv(f"lastfm_genres_2024_3.csv")
save_df5 = pd.read_csv(f"lastfm_genres_2024_4.csv")
save_df6 = pd.read_csv(f"lastfm_genres_2024_5.csv")
save_df7 = pd.read_csv(f"lastfm_genres_2024_6.csv")
save_df8 = pd.read_csv(f"lastfm_genres_2024_7.csv")
save_df9 = pd.read_csv(f"lastfm_genres_2024_8.csv")
save_df10 = pd.read_csv(f"lastfm_genres_2024_9.csv")
save_df11 = pd.read_csv(f"lastfm_genres_2024_10.csv")
save_df12 = pd.read_csv(f"lastfm_genres_2024_11.csv")
save_df13 = pd.read_csv(f"lastfm_genres_2024_12.csv")
save_df14 = pd.read_csv(f"lastfm_genres_2024_14.csv")
save_df15 = pd.read_csv(f"lastfm_genres_2024_16.csv")
save_df16 = pd.read_csv(f"lastfm_genres_2024_18.csv")
df_total_checked = pd.concat([save_df1, save_df2, save_df3, save_df4, save_df5, save_df6, save_df7, save_df8, save_df9, save_df10, save_df11, save_df12, save_df13, save_df14, save_df15, save_df16]).drop_duplicates().reset_index(drop=True)

In [12]:
df_total_checked = df_total_checked.drop_duplicates(subset='track_id').reset_index(drop=True)
df_total_checked

,track_id,track_name,artist_name,new_popularity_2025,album,popularity_2024,duration_ms,explicit,release_date,af_danceability,...,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,genres
0,3Vj8RQkFwOmDvUrwu6MtQZ,"Yo Love - From ""Queen & Slim: The Soundtrack""",Vince Staples,55,"Yo Love (From ""Queen & Slim: The Soundtrack"")",62,143830,True,2019-11-11,0.703,...,0,-9.032,1,0.0724,0.5650,0.000003,0.0987,0.491,147.038,Unknown
1,5jEtmndo1eO9yoKVSHvMpz,Hospedaje,Sebas,23,Hospedaje,26,208500,False,2019-11-08,0.747,...,11,-3.836,0,0.1130,0.0617,0.000038,0.3070,0.557,180.074,"pop, latino"
2,1YP7UWFugY2xRX3PdieTAs,Himno de Fe y Alegría,Jairo y Su Barra Roja,26,Homenaje Musical al Fútbol: Selección Colombia,22,344733,False,2015-06-23,0.859,...,11,-5.698,1,0.0423,0.0396,0.016000,0.2710,0.851,109.759,Unknown
3,1jt2raacDSDXbyN9o7NWcl,Qué Bien Te Ves,Kexxy Pardo,11,Qué Bien Te Ves,12,200816,False,2019-11-07,0.769,...,1,-3.752,1,0.1170,0.2420,0.001650,0.1560,0.765,98.003,Unknown
4,2TDgoPhthPcoy1mYcQxS8P,Lambaláda,Bert and Friends,0,Supr,0,220041,False,2019-11-10,0.719,...,7,-4.332,0,0.0325,0.3930,0.021800,0.2430,0.367,116.999,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198072,4EdKJoJqBr73rA7aqrdBA5,Diz,Gabriela Rocha,54,Diz,57,275306,False,2019-11-07,0.481,...,4,-6.854,0,0.0260,0.5940,0.000005,0.1130,0.171,73.989,Unknown
198073,3jdRcpohkvbG7WPo3mjiZD,Cross the Line,Dubioza kolektiv,21,Cross the Line,23,238064,False,2019-11-05,0.620,...,2,-6.097,1,0.0306,0.1860,0.000010,0.2520,0.965,123.977,Unknown
198074,0RMmzG0f5szIxC49oucu8w,Picasso (feat. Gunna),Leeky Bandz,19,Picasso (feat. Gunna),23,192000,True,2019-11-08,0.820,...,1,-6.272,1,0.1590,0.1070,0.000000,0.1120,0.127,140.003,Unknown
198075,3V0nnQhqvbE3JmiDdnzQFQ,July (feat. Leon Bridges),Noah Cyrus,68,July (feat. Leon Bridges),74,152280,False,2019-11-08,0.708,...,9,-9.603,0,0.0424,0.8530,0.000000,0.1310,0.362,72.500,"indie pop, intricate melodic phrasing, extensi..."


In [13]:
# Realizar el merge de df con df_total_checked basado en todos los campos excepto 'genres'
merged_df = df.merge(df_total_checked[['track_id', 'genres']], on='track_id', how='left', suffixes=('', '_new'))

# Sustituir df["genres"] por df_total_checked["genres"] donde coinciden todos los demás campos
merged_df['genres'] = merged_df['genres_new'].combine_first(merged_df['genres'])

# Eliminar la columna 'genres_new' que se creó durante el merge
merged_df.drop(columns=['genres_new'], inplace=True)

# Actualizar df con los valores de merged_df
df_full = merged_df

df_full

,track_id,track_name,artist_name,new_popularity_2025,album,popularity_2024,duration_ms,explicit,release_date,af_danceability,...,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,genres
0,3Vj8RQkFwOmDvUrwu6MtQZ,"Yo Love - From ""Queen & Slim: The Soundtrack""",Vince Staples,55,"Yo Love (From ""Queen & Slim: The Soundtrack"")",62,143830,True,2019-11-11,0.703,...,0,-9.032,1,0.0724,0.5650,0.000003,0.0987,0.491,147.038,Unknown
1,5jEtmndo1eO9yoKVSHvMpz,Hospedaje,Sebas,23,Hospedaje,26,208500,False,2019-11-08,0.747,...,11,-3.836,0,0.1130,0.0617,0.000038,0.3070,0.557,180.074,"pop, latino"
2,1YP7UWFugY2xRX3PdieTAs,Himno de Fe y Alegría,Jairo y Su Barra Roja,26,Homenaje Musical al Fútbol: Selección Colombia,22,344733,False,2015-06-23,0.859,...,11,-5.698,1,0.0423,0.0396,0.016000,0.2710,0.851,109.759,Unknown
3,1jt2raacDSDXbyN9o7NWcl,Qué Bien Te Ves,Kexxy Pardo,11,Qué Bien Te Ves,12,200816,False,2019-11-07,0.769,...,1,-3.752,1,0.1170,0.2420,0.001650,0.1560,0.765,98.003,Unknown
4,2TDgoPhthPcoy1mYcQxS8P,Lambaláda,Bert and Friends,0,Supr,0,220041,False,2019-11-10,0.719,...,7,-4.332,0,0.0325,0.3930,0.021800,0.2430,0.367,116.999,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198121,4EdKJoJqBr73rA7aqrdBA5,Diz,Gabriela Rocha,54,Diz,57,275306,False,2019-11-07,0.481,...,4,-6.854,0,0.0260,0.5940,0.000005,0.1130,0.171,73.989,Unknown
198122,3jdRcpohkvbG7WPo3mjiZD,Cross the Line,Dubioza kolektiv,21,Cross the Line,23,238064,False,2019-11-05,0.620,...,2,-6.097,1,0.0306,0.1860,0.000010,0.2520,0.965,123.977,Unknown
198123,0RMmzG0f5szIxC49oucu8w,Picasso (feat. Gunna),Leeky Bandz,19,Picasso (feat. Gunna),23,192000,True,2019-11-08,0.820,...,1,-6.272,1,0.1590,0.1070,0.000000,0.1120,0.127,140.003,Unknown
198124,3V0nnQhqvbE3JmiDdnzQFQ,July (feat. Leon Bridges),Noah Cyrus,68,July (feat. Leon Bridges),74,152280,False,2019-11-08,0.708,...,9,-9.603,0,0.0424,0.8530,0.000000,0.1310,0.362,72.500,"indie pop, intricate melodic phrasing, extensi..."


In [15]:
df_full.to_csv("data_genres_pre_2024.csv", index=False)

In [15]:
df1 = pd.read_csv(f"last_fm_checked_01.csv")
df2  = pd.read_csv(f"lastfm_genres_17.csv")
df3 = pd.read_csv(f"lastfm_genres_18.csv")
df4 = pd.read_csv(f"lastfm_genres_19.csv")
df5 = pd.read_csv(f"lastfm_genres_20.csv")
df6 = pd.read_csv(f"lastfm_genres_21.csv")
df7 = pd.read_csv(f"lastfm_genres_22.csv")
df8 = pd.read_csv(f"lastfm_genres_23.csv")
df9 = pd.read_csv(f"lastfm_genres_24.csv")

df_total_checked = pd.concat([df1, df2,df6,df9,df8,df7, df3,df4, df5]).drop_duplicates().reset_index(drop=True)

In [16]:
# Crear un conjunto de tuplas (artista, nombre) de save_df para una búsqueda rápida
save_df_set = set(zip(df_total_checked['artists'], df_total_checked['name']))

# Filtrar artists_songs para eliminar las filas que están en save_df_set
artists_songs_filtered = artists_songs[~artists_songs.apply(lambda row: (row['artists'], row['name']) in save_df_set, axis=1)]

artists_songs_filtered


,artists,name,genres


In [17]:
#test_df

In [18]:
# Obtener todos los géneros en una lista
all_genres = df_total_checked['genres'].str.split(', ').explode()

# Convertir la lista a un conjunto para obtener géneros únicos
unique_genres = set(all_genres)

print(len(unique_genres))

24860


In [19]:
#df_total_checked.to_csv("lastfm_genres_total_01.csv", index=False)

In [67]:
df_completo = pd.read_csv("last_fm_checked_completo.csv")
df_completo



,artists,name,genres
0,Olivia Rodrigo,drivers license,"pop, bedroom pop, sad, 2021, Ballad"
1,Ariana Grande,positions,"pop, rnb, 2020, 2020s, trap"
2,"24kGoldn, iann dior",Mood (feat. iann dior),"pop, rap, pop rap, 2020, 2020s"
3,"Bad Bunny, Jhay Cortez",DÁKITI,"rnb, latin, 2020s, rap, ambient pop"
4,KAROL G,BICHOTA,"Reggaeton, latin, 2020, 2020s, Karol G"
...,...,...,...
157679,"Georgette Heyer, Irina Salkow",Kapitel 174 - Der Page und die Herzogin,Unknown
157680,Aschberger Saitenmusi,Weissach Boarisch,Unknown
157681,Carmen Miranda,Amor Ideal,Unknown
157682,Carlos Gardel,Alma de loca,tango


In [68]:
df



,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
19405,0.721,Olivia Rodrigo,0.585,242014,0.436,1,7lPN2DXiMsVn7XUKtOW1CS,0.000013,10,0.1050,-8.761,1,drivers license,100,2021-01-08,0.0601,143.874,0.132,2021
19228,0.468,Ariana Grande,0.737,172325,0.802,1,35mvY5S1H3J2QZyna3TFe0,0.000000,0,0.0931,-4.771,1,positions,96,2020-10-30,0.0878,144.015,0.682,2020
19225,0.221,"24kGoldn, iann dior",0.700,140526,0.722,1,3tjFYV6RSFtuktYl3ZtYcq,0.000000,7,0.2720,-3.558,0,Mood (feat. iann dior),96,2020-07-24,0.0369,90.989,0.756,2020
19320,0.212,KAROL G,0.863,178947,0.666,1,7vrJn5hDSXRmdXoR30KgF1,0.000493,1,0.1030,-4.158,0,BICHOTA,95,2020-10-23,0.1520,163.908,0.838,2020
19242,0.401,"Bad Bunny, Jhay Cortez",0.731,205090,0.573,1,47EiUVwUp4C9fGccaPuUCS,0.000052,4,0.1130,-10.059,0,DÁKITI,95,2020-10-30,0.0544,109.928,0.145,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170296,0.107,Don Omar,0.886,317280,0.708,0,7u6U4fGBmcDSYpFPMW5YlQ,0.011400,4,0.0440,-5.915,0,Salió El Sol,0,2021-01-22,0.0911,109.976,0.965,2021
170297,0.217,Don Omar,0.741,255680,0.787,0,7wPssXinLiw4KoSpdQVpyn,0.017900,5,0.0835,-6.228,1,Pobre Diabla,0,2021-01-22,0.0525,91.993,0.771,2021
126213,0.990,Hesen Zîrek,0.266,387657,0.660,0,25eqAVo21l5HhveZOt51iZ,0.754000,7,0.8250,-8.179,1,Eman Eî Yar,0,1951-06-02,0.0761,93.597,0.434,1951
126183,0.994,Suresh Wadkar,0.568,366907,0.260,0,20Ct8EHjHLEW4iWlE49Fyz,0.869000,1,0.1610,-14.509,1,Koti Koti Roope Tuzi,0,1951-12-01,0.0468,113.708,0.706,1951


In [ ]:
# Crear un conjunto de tuplas (artista, nombre) de df_completo para una búsqueda rápida
df_completo_set = set(zip(df_completo['artists'], df_completo['name']))

# Filtrar df para eliminar las filas que no están en df_completo_set
df_filtered = df[~df.apply(lambda row: (row['artists'], row['name']) in df_completo_set, axis=1)]

df_filtered


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year


In [ ]:
import re
import pandas as pd
df_completo = pd.read_csv("last_fm_checked_completo.csv")


df_completo['main_genre'] = ""
df_completo['subgenre_1'] = ""
df_completo['subgenre_2'] = ""

for row in df_completo.itertuples():
    generos = row.genres.split(", ")
    filtrado = []
    for gen in generos:
        if (gen.lower() in row.artists.lower()) or (gen.lower() in ["","vugube62", "title is a full sentence","names", "british i like", "j cole", "somafm", "tantotempotaste","myspotigrambot", "gilles peterson", "paul chambers", "like", "new york"]):
            gen = ""
        else:
            filtrado.append(gen)

    df_completo.at[row.Index, 'genres'] = ", ".join(filtrado)


diccionario_recuento_gen = {}
for index, row in df_completo.iterrows():
    lista_genres = row["genres"].lower().split(", ")
    for genre in lista_genres:
        if genre in diccionario_recuento_gen:
            diccionario_recuento_gen[genre] += 1
        else:
            diccionario_recuento_gen[genre] = 1

sorted_genres = sorted(diccionario_recuento_gen.items(), key=lambda item: item[1], reverse=True)


for row in df_completo.itertuples():
    generos = row.genres.split(", ")
    filtrado = []
    for gen in generos:
        if (diccionario_recuento_gen[gen.lower()] < 61) or (re.match(r'\d{4,}|\d+s|-?\d+', gen)) :
            gen = ""
        else:
            filtrado.append(gen)

    df_completo.at[row.Index, 'genres'] = ", ".join(filtrado)

In [ ]:



longest_genres_list = df_completo['genres'].str.split(', ').apply(len).idxmax()
print(df_completo.loc[longest_genres_list, 'genres'])


diccionario_recuento_gen2 = {}
for index, row in df_completo.iterrows():
    lista_genres = row["genres"].lower().split(", ")
    for genre in lista_genres:
        if genre in diccionario_recuento_gen2:
            diccionario_recuento_gen2[genre] += 1
        else:
            diccionario_recuento_gen2[genre] = 1

sorted_genres2 = sorted(diccionario_recuento_gen2.items(), key=lambda item: item[1], reverse=True)
print(sorted_genres2)

df_completo['genres'] = df_completo['genres'].apply(lambda x: "Unknown" if x == "" else x)

unknown_count = df_completo['genres'].apply(lambda x: x == "Unknown").sum()
print(f"Unknown or empty count: {unknown_count}")

dance, latin, Reggaeton, latin pop, pop
[('unknown', 84467), ('rock', 17581), ('pop', 10160), ('classic rock', 8323), ('jazz', 7726), ('soul', 5901), ('female vocalists', 5818), ('country', 5367), ('alternative', 5039), ('hip-hop', 4151), ('oldies', 4073), ('hard rock', 3924), ('rap', 3661), ('folk', 3504), ('singer-songwriter', 3161), ('alternative rock', 3147), ('instrumental', 3130), ('blues', 2996), ('hip hop', 2922), ('indie', 2883), ('rnb', 2790), ('metal', 2241), ('funk', 2229), ('punk', 2210), ('christmas', 2194), ('soundtrack', 2136), ('cover', 2054), ('electronic', 2043), ('dance', 1997), ('new wave', 1904), ('piano', 1792), ('heavy metal', 1692), ('easy listening', 1649), ('love', 1634), ('progressive rock', 1517), ('latin', 1474), ('punk rock', 1436), ('fip', 1316), ('british', 1306), ('blues rock', 1298), ('acoustic', 1266), ('classic country', 1217), ('classical', 1203), ('indie rock', 1202), ('swing', 1185), ('jazz fusion', 1146), ('soft rock', 1100), ('psychedelic', 101

In [87]:
for i, row in df_completo.iterrows():
    genres = row['genres'].lower().split(", ")
    generos_por_recuento = sorted(genres, key=lambda x: diccionario_recuento_gen[x], reverse=True)
    if len(genres) == 1:
        df_completo.at[i, 'main_genre'] = generos_por_recuento[0]
        df_completo.at[i, 'subgenre_1'] = "None"
        df_completo.at[i, 'subgenre_2'] = "None"
    if len(genres) == 2:
        df_completo.at[i, 'main_genre'] = generos_por_recuento[0]
        df_completo.at[i, 'subgenre_1'] = generos_por_recuento[1]
        df_completo.at[i, 'subgenre_2'] = "None"
    if len(genres) > 2:
        df_completo.at[i, 'main_genre'] = generos_por_recuento[0]
        df_completo.at[i, 'subgenre_1'] = generos_por_recuento[1]
        df_completo.at[i, 'subgenre_2'] = generos_por_recuento[2]


df_completo


,artists,name,genres,main_genre,subgenre_1,subgenre_2
0,Olivia Rodrigo,drivers license,"pop, sad, Ballad",pop,ballad,sad
1,Ariana Grande,positions,"pop, rnb, trap",pop,rnb,trap
2,"24kGoldn, iann dior",Mood (feat. iann dior),"pop, rap, pop rap",pop,rap,pop rap
3,"Bad Bunny, Jhay Cortez",DÁKITI,"rnb, latin, rap",rap,rnb,latin
4,KAROL G,BICHOTA,"Reggaeton, latin",latin,reggaeton,None
...,...,...,...,...,...,...
157679,"Georgette Heyer, Irina Salkow",Kapitel 174 - Der Page und die Herzogin,Unknown,unknown,None,None
157680,Aschberger Saitenmusi,Weissach Boarisch,Unknown,unknown,None,None
157681,Carmen Miranda,Amor Ideal,Unknown,unknown,None,None
157682,Carlos Gardel,Alma de loca,tango,tango,None,None


In [ ]:
#df_completo.to_csv("canciones_generos.csv", index=False)

In [ ]:
"""
unknown_count = df_completo['genres'].str.contains('Unknown').sum()
error_count = df_completo['genres'].str.contains('Error').sum()

print(f"Unknown count: {unknown_count}")
print(f"Error count: {error_count}")


df_completo['genres'].value_counts()
diccionario_recuento_gen = {}

for index, row in df_completo.iterrows():
    lista_genres = row["genres"].lower().split(", ")
    for genre in lista_genres:
        if genre in diccionario_recuento_gen:
            diccionario_recuento_gen[genre] += 1
        else:
            diccionario_recuento_gen[genre] = 1


# Ordenar las claves del diccionario en orden decreciente por sus valores
sorted_genres = sorted(diccionario_recuento_gen.items(), key=lambda item: item[1], reverse=True)

# Imprimir los géneros ordenados
i = 0
print(len(sorted_genres))
for genre, count in sorted_genres:
    if count > 60:
        if not (re.match(r'\d{4,}|\d+s|-?\d+', genre)):

            print(f"{genre}: {count}")
            i += 1
print(i)


for row in df_completo.itertuples():
    generos = row.genres.split(", ")
    filtrado = []
    for gen in generos:
        if (gen.lower() in row.artists.lower()) or (gen.lower() in ["","vugube62", "title is a full sentence","names", "british i like", "j cole", "somafm", "tantotempotaste", "gilles peterson", "paul chambers", "like", "new york"]):
            gen = ""
        else:
            filtrado.append(gen)

    df_completo.at[row.Index, 'genres'] = ", ".join(filtrado)


df_completo
"""

Unknown count: 78493
Error count: 0
429
unknown: 78497
rock: 17581
pop: 10160
classic rock: 8323
jazz: 7726
: 5970
soul: 5901
female vocalists: 5818
country: 5367
alternative: 5039
hip-hop: 4151
oldies: 4073
hard rock: 3924
rap: 3661
folk: 3504
singer-songwriter: 3161
alternative rock: 3147
instrumental: 3130
blues: 2996
hip hop: 2922
indie: 2883
rnb: 2790
metal: 2241
funk: 2229
punk: 2210
christmas: 2194
soundtrack: 2136
cover: 2054
electronic: 2043
dance: 1997
new wave: 1904
piano: 1792
heavy metal: 1692
easy listening: 1649
love: 1634
progressive rock: 1517
latin: 1474
punk rock: 1436
fip: 1316
british: 1306
blues rock: 1298
acoustic: 1266
classic country: 1217
classical: 1203
indie rock: 1202
swing: 1185
jazz fusion: 1146
soft rock: 1100
psychedelic: 1011
disco: 1002
reggae: 989
guitar: 984
male vocalists: 944
ballad: 907
post-punk: 904
spanish: 886
psychedelic rock: 832
folk rock: 801
beautiful: 789
pop rock: 783
chillout: 765
motown: 731
mellow: 730
saxophone: 726
vocal jazz: 722

,artists,name,genres
0,Olivia Rodrigo,drivers license,"pop, sad, Ballad"
1,Ariana Grande,positions,"pop, rnb, trap"
2,"24kGoldn, iann dior",Mood (feat. iann dior),"pop, rap, pop rap"
3,"Bad Bunny, Jhay Cortez",DÁKITI,"rnb, latin, rap"
4,KAROL G,BICHOTA,"Reggaeton, latin"
...,...,...,...
157679,"Georgette Heyer, Irina Salkow",Kapitel 174 - Der Page und die Herzogin,Unknown
157680,Aschberger Saitenmusi,Weissach Boarisch,Unknown
157681,Carmen Miranda,Amor Ideal,Unknown
157682,Carlos Gardel,Alma de loca,tango


In [ ]:
"""df_completo[df_completo['artists'].str.lower().str.contains('j. cole')]"""

,artists,name,genres
187,J. Cole,No Role Modelz,"Hip-Hop, rap, hip hop"
400,J. Cole,MIDDLE CHILD,"hip hop, rap, trap"
685,J. Cole,Work Out,"rap, hip hop, Hip-Hop"
712,J. Cole,Wet Dreamz,"hip hop, rap, Hip-Hop"
1680,"6LACK, J. Cole",Pretty Little Fears (feat. J. Cole),"Love, trap, pop rap, cloud rap, alternative rnb"
...,...,...,...
34305,J. Cole,Chaining Day,"hip hop, love at first listen, pop rap"
34379,"Bas, J. Cole, KQuick",Lit - Bonus,
35177,J. Cole,Runaway,"Hip-Hop, hip hop, rap"
37983,J. Cole,Dollar And A Dream III,"Hip-Hop, rap, old school"


In [ ]:
df_spotify = pd.read_csv("spotify_artists_genres_11.csv")
df_spotify

df_spotify['gen_artist'] = df_spotify['gen_artist'].apply(lambda x: eval(x) if isinstance(x, str) else x)


### Agregar géneros de Spotify a las canciones de Last.fm TAREA

def merge_genres(row):
    spotify_genres = df_spotify.loc[df_spotify['name_artist'] in row['artists'], 'gen_artist']
    if not spotify_genres.empty:
        spotify_genres_list = spotify_genres.iloc[0]
        if isinstance(spotify_genres_list, list):
            combined_genres = row['genres'].split(', ') + spotify_genres_list
            return ', '.join(set(combined_genres))
    return row['genres']

df_completo['genres'] = df_completo.apply(merge_genres, axis=1)

,artists,name,genres,main_genre,subgenre_1,subgenre_2
0,Olivia Rodrigo,drivers license,"pop, sad, Ballad",pop,ballad,sad
1,Ariana Grande,positions,"pop, rnb, trap",pop,rnb,trap
2,"24kGoldn, iann dior",Mood (feat. iann dior),"pop, rap, pop rap",pop,rap,pop rap
3,"Bad Bunny, Jhay Cortez",DÁKITI,"rnb, latin, rap",rap,rnb,latin
4,KAROL G,BICHOTA,"Reggaeton, latin",latin,reggaeton,None
...,...,...,...,...,...,...
157679,"Georgette Heyer, Irina Salkow",Kapitel 174 - Der Page und die Herzogin,Unknown,unknown,None,None
157680,Aschberger Saitenmusi,Weissach Boarisch,Unknown,unknown,None,None
157681,Carmen Miranda,Amor Ideal,Unknown,unknown,None,None
157682,Carlos Gardel,Alma de loca,tango,tango,None,None


In [92]:
df_spotify

,name_artist,gen_artist
0,Sergio Marini & Luke,['Faltante']
1,Makiko,['Faltante']
2,Naushad,['malayalam pop']
3,S27,['Faltante']
4,Emil Lundin,['Faltante']
...,...,...
31951,Cri-Cri,"['reggaeton chileno', 'chilean trap', 'chilean..."
31952,Rogelio Martinez El Rm,"['corrido', 'corridos bélicos', 'banda']"
31953,Good Rockin Charles,"['blues', 'classic blues']"
31954,Trin-I-Tee 5:7,"['gospel', 'gospel r&b', 'worship']"
